In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from pathlib import Path

import pandas as pd
import xarray as xr
import pandas_indexing.accessors
from aneris.harmonize import Harmonizer
from aneris.downscaling import Downscaler
from aneris.grid import Gridder
from pandas import DataFrame
from pandas_indexing import isin, semijoin

from concordia import VariableDefinitions, RegionMapping, combine_countries

In [3]:
# Potentially better gridding performance??
# from dask.distributed import Client
# client = Client()

# Read model and historic data including overrides

In [4]:
base_path = Path(
    "/Users/coroa/Library/CloudStorage/OneDrive-SharedLibraries-IIASA/RESCUE - WP 1/data"
)
data_path = Path("../data")
base_year = 2015 # in which year scenario data should be harmonized to historical data
country_combinations = {
    "sdn_ssd": ["ssd", "sdn"],
    "isr_pse": ["isr", "pse"],
    "srb_ksv": ["srb", "srb (kosovo)"]
}

## Variable definition files

The variable definition file is a CSV or yaml file that needs to contain the `variable`-name, its `sector`, `gas` components and whether it is expected `global` (or regional instead).

Here we generate one based on the cmip6 historical data we have that could be used as a basis but we would want to finetune this by hand.

In [5]:
variabledefs = VariableDefinitions.from_csv(data_path / "variabledefs-cmip6.csv")
variabledefs.data.head()

,,,unit,global
variable,gas,sector,,
CEDS+|9+ Sectors|Emissions|BC|Agricultural Waste Burning|Unharmonized,BC,Agricultural Waste Burning,Mt BC/yr,False
CEDS+|9+ Sectors|Emissions|BC|Agriculture|Unharmonized,BC,Agriculture,Mt BC/yr,False
CEDS+|9+ Sectors|Emissions|BC|Aircraft|Unharmonized,BC,Aircraft,Mt BC/yr,True
CEDS+|9+ Sectors|Emissions|BC|Energy Sector|Unharmonized,BC,Energy Sector,Mt BC/yr,False
CEDS+|9+ Sectors|Emissions|BC|Forest Burning|Unharmonized,BC,Forest Burning,Mt BC/yr,False


## RegionMapping helps reading in a region definition file

In [6]:
regionmapping = RegionMapping.from_regiondef(
    base_path / "historical/cmip6/remind_region_mapping.csv"
)
regionmapping.data = combine_countries(regionmapping.data, **country_combinations, agg_func="first")

## Model and historic data read in

Can be read in and prepared using `read_iamc` or the `variabledefs`

In [12]:
hist = (
    pd.read_csv(base_path / "historical/cmip6/history.csv", index_col=list(range(5)))
    .rename_axis(index={"Region": "Country"})
    .pipe(variabledefs.load_data, levels=["country", "gas", "sector", "unit"])
    .pipe(combine_countries, **country_combinations)
)
hist.head()

year                                                 1990      1991      1992   
country gas sector                     unit                                     
World   BC  Agricultural Waste Burning Mt BC/yr  0.199538  0.221455  0.183570  \
            Agriculture                Mt BC/yr  0.000000  0.000000  0.000000   
            Aircraft                   Mt BC/yr  0.007039  0.006816  0.006882   
            Energy Sector              Mt BC/yr  0.544495  0.530507  0.539325   
            Forest Burning             Mt BC/yr  0.470999  0.723609  0.406275   

year                                                 1993      1994      1995   
country gas sector                     unit                                     
World   BC  Agricultural Waste Burning Mt BC/yr  0.201017  0.218786  0.202448  \
            Agriculture                Mt BC/yr  0.000000  0.000000  0.000000   
            Aircraft                   Mt BC/yr  0.006974  0.007298  0.007482   
            Energy Sector              Mt BC/yr  0.569899  0.633019  0.704017   
            Forest Burning             Mt BC/yr  0.520459  0.749207  0.702724   

year                                                 1996      1997      1998   
country gas sector                     unit                                     
World   BC  Agricultural Waste Burning Mt BC/yr  0.199741  0.211430  0.248320  \
            Agriculture                Mt BC/yr  0.000000  0.000000  0.000000   
            Aircraft                   Mt BC/yr  0.007828  0.008130  0.008314   
            Energy Sector              Mt BC/yr  0.705651  0.715599  0.681604   
            Forest Burning             Mt BC/yr  0.478484  0.703453  0.962960   

year                                                 1999  ...      2006   
country gas sector                     unit                ...             
World   BC  Agricultural Waste Burning Mt BC/yr  0.179020  ...  0.218279  \
            Agriculture                Mt BC/yr  0.000000  ...  0.000000   
            Aircraft                   Mt BC/yr  0.008608  ...  0.009826   
            Energy Sector              Mt BC/yr  0.652070  ...  1.001480   
            Forest Burning             Mt BC/yr  0.520379  ...  0.526123   

year                                                 2007      2008      2009   
country gas sector                     unit                                     
World   BC  Agricultural Waste Burning Mt BC/yr  0.189355  0.228403  0.187130  \
            Agriculture                Mt BC/yr  0.000000  0.000000  0.000000   
            Aircraft                   Mt BC/yr  0.010087  0.009983  0.009447   
            Energy Sector              Mt BC/yr  1.059195  1.014806  1.057247   
            Forest Burning             Mt BC/yr  0.621482  0.402895  0.340771   

year                                                 2010      2011      2012   
country gas sector                     unit                                     
World   BC  Agricultural Waste Burning Mt BC/yr  0.180565  0.160563  0.158698  \
            Agriculture                Mt BC/yr  0.000000  0.000000  0.000000   
            Aircraft                   Mt BC/yr  0.009898  0.010158  0.010287   
            Energy Sector              Mt BC/yr  1.099077  1.180117  1.193312   
            Forest Burning             Mt BC/yr  0.648860  0.360724  0.543624   

year                                                 2013      2014      2015  
country gas sector                     unit                                    
World   BC  Agricultural Waste Burning Mt BC/yr  0.145540  0.184817  0.187327  
            Agriculture                Mt BC/yr  0.000000  0.000000  0.000000  
            Aircraft                   Mt BC/yr  0.010415  0.010468  0.010672  
            Energy Sector              Mt BC/yr  1.219606  1.191399  1.190588  
            Forest Burning             Mt BC/yr  0.412669  0.540295  0.497394  

[5 rows x 26 columns]

In [13]:
model = (
    pd.read_excel(
        base_path / "iam_files/cmip6/REMIND-MAGPIE_SSP5-34-OS/ssp5-34-os.xlsx",
        sheet_name="data",
        index_col=list(range(5)),
    )
    .rename(index={"Mt CO2-equiv/yr": "Mt CO2-eq/yr"}, level="Unit")
    .pipe(
        variabledefs.load_data,
        levels=["model", "scenario", "region", "gas", "sector", "unit"],
        ignore_missing=True,
    )
)
model.head()

year                                                                           2005   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0439  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1055   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0387   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0817   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2010   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0633  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1164   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0384   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0841   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2015   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0799  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1243   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0892   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2020   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0966  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1322   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0943   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2030   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1279  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1400   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0369   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1054   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2040   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1568  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1509   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0357   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1061   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2050   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1768  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.

In [14]:
harm_overrides = (
    pd.read_excel(
        base_path / "iam_files/cmip6/REMIND-MAGPIE_SSP5-34-OS/ssp5-34-os.xlsx",
        sheet_name="harmonization",
        index_col=list(range(4)),
        usecols=list(range(5)),
    )
    .rename_axis(index=str.lower)
    .rename(columns=str.lower)
    .pipe(
        variabledefs.load_data,
        ignore_missing=True,
        levels=["region", "gas", "sector"],
        timeseries=False,
    )
    .method
)

harm_overrides.head()

region  gas  sector        
CHN     BC   Energy Sector     reduce_offset_2150
AFR     BC   Forest Burning        constant_ratio
EUR     BC   Forest Burning        constant_ratio
ROW     BC   Forest Burning        constant_ratio
USA     BC   Forest Burning        constant_ratio
Name: method, dtype: object

In [15]:
model.head()

year                                                                           2005   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0439  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1055   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0387   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0817   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2010   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0633  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1164   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0384   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0841   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2015   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0799  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1243   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0892   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2020   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.0966  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1322   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0383   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.0943   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2030   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1279  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1400   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0369   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1054   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2040   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1568  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.1509   
                             EUR    BC  Agricultural Waste Burning Mt BC/yr  0.0357   
                             IND    BC  Agricultural Waste Burning Mt BC/yr  0.1061   
                             JPN    BC  Agricultural Waste Burning Mt BC/yr  0.0000   

year                                                                           2050   
model         scenario       region gas sector                     unit               
REMIND-MAGPIE SSP5-34-OS-V27 AFR    BC  Agricultural Waste Burning Mt BC/yr  0.1768  \
                             CHN    BC  Agricultural Waste Burning Mt BC/yr  0.

# Harmonization

## Preparation of input data

In [17]:
hist_agg = pd.concat(
    [
        hist.pix.semijoin(variabledefs.index_regional, how="inner").pipe(
            regionmapping.aggregate
        ),
        hist.pix.semijoin(variabledefs.index_global, how="inner")
        .loc[isin(country="World")]
        .rename_axis(index={"country": "region"}),
    ]
)

In [18]:
model_agg = pd.concat(
    [
        model.pix.semijoin(variabledefs.index_regional, how="inner").loc[
            isin(region=regionmapping.data.unique())
        ],
        model.pix.semijoin(variabledefs.index_global, how="inner").loc[
            isin(region="World")
        ],
    ]
)

## Harmonize all model, scenarios combinations

In [19]:
harmonized = []
for m, s in model.index.pix.project(["model", "scenario"]).unique():
    scen = model_agg.loc[isin(model=m, scenario=s)].droplevel(["model", "scenario"])
    h = Harmonizer(
        scen, hist_agg.pix.semijoin(scen.index, how="right"), harm_idx=scen.index.names
    )
    result = h.harmonize(year=base_year, overrides=harm_overrides)
    harmonized.append(result.pix.assign(model=m, scenario=s))
harmonized = pd.concat(harmonized)

# TODO harmonization casts columns to str!!
harmonized = harmonized.rename(columns=int)
harmonized.head()

/Users/coroa/repos/aneris/src/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))
INFO:root:Harmonizing with reduce_offset_2150_cov
INFO:root:Harmonizing with model_zero
INFO:root:Harmonizing with hist_zero
INFO:root:Harmonizing with constant_ratio
INFO:root:Harmonizing with reduce_offset_2150
INFO:root:Harmonizing with reduce_ratio_2080
INFO:root:Harmonizing with constant_offset
INFO:root:Harmonizing with reduce_offset_2100


year                                                                             2015   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.058867  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.013035   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.004719   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.005197   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2020   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.076346  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.025056   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.005963   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.013408   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2030   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.109204  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.041098   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.007051   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.030730   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2040   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.139662  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.060239   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.008338   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.037653   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2050   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.161220  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.094981   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.009125   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.040575   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2060   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.146278  \
                               CHN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.085123   
                               EUR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.011613   
                               IND    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.047498   
                               JPN    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0.000437   

year                                                                             2070   
gas sector                     region unit     model         scenario                   
BC  Agricultural Waste Burning AFR    Mt BC/yr REMIND-MAGPIE SSP5-34-OS-V27  0

# Downscaling

## Prepare GDP proxy

Read in different GDP scenarios for SSP1 to SSP5 from SSP DB.

In [67]:
gdp = (
    pd.read_csv(
        base_path / "historical" / "SspDb_country_data_2013-06-12.csv",
        index_col=list(range(5)),
    )
    .rename_axis(index=str.lower)
    .loc[
        isin(
            model="OECD Env-Growth",
            scenario=[f"SSP{n+1}_v9_130325" for n in range(5)],
            variable="GDP|PPP",
        )
    ]
    .dropna(how="all", axis=1)
    .rename_axis(index={"scenario": "ssp", "region": "country"})
    .rename(index=str.lower, level="country")
    .rename(columns=int)
    .pix.project(["ssp", "country"])
    .pipe(combine_countries, **country_combinations)
)
gdp.head()

2000     2005     2010     2015     2020     2025   
ssp            country                                                         
SSP1_v9_130325 abw        0.000    0.128    0.151    0.205    0.266    0.357  \
               afg        0.000   22.372   37.237   47.643   59.153   78.506   
               ago       34.484   55.315   98.686  127.295  171.984  203.363   
               alb       14.743   19.170   24.545   26.919   30.452   35.126   
               are      209.548  272.055  318.142  439.900  536.362  652.339   

                           2030     2035      2040      2045  ...      2055   
ssp            country                                        ...             
SSP1_v9_130325 abw        0.492    0.679     0.926     1.237  ...     2.067  \
               afg      109.567  157.250   227.005   324.841  ...   627.328   
               ago      232.320  261.469   304.102   369.997  ...   591.431   
               alb       41.037   48.034    55.608    63.289  ...    76.939   
               are      787.776  927.181  1061.287  1176.838  ...  1343.919   

                            2060      2065      2070      2075      2080   
ssp            country                                                     
SSP1_v9_130325 abw         2.586     3.165     3.797     4.470     5.171  \
               afg       839.381  1094.144  1389.577  1722.005  2086.194   
               ago       750.860   942.049  1160.284  1400.116  1655.046   
               alb        83.030    88.331    92.473    95.687    98.004   
               are      1421.730  1487.358  1535.672  1560.609  1569.904   

                            2085      2090      2095      2100  
ssp            country                                          
SSP1_v9_130325 abw         5.842     6.469     7.034     7.505  
               afg      2474.187  2878.709  3293.204  3707.958  
               ago      1917.629  2181.823  2441.164  2687.836  
               alb        99.153    99.277    98.596    97.006  
               are      1565.244  1550.719  1526.111  1486.250  

[5 rows x 21 columns]

Determine likely SSP for each harmonized pathway from scenario string and create proxy data aligned with pathways

In [68]:
SSP_per_pathway = (
    harmonized.index.pix.project(["model", "scenario"])
    .unique()
    .to_frame()
    .scenario.str.extract("(SSP[1-5])")[0]
    .fillna("SSP2")
)
gdp = semijoin(
    gdp,
    SSP_per_pathway.index.pix.assign(ssp=SSP_per_pathway + "_v9_130325"),
    how="right",
).pix.project(["model", "scenario", "country"])
gdp.head()

2000     2005     2010     2015   
model         scenario       country                                       
REMIND-MAGPIE SSP5-34-OS-V27 abw        0.000    0.128    0.151    0.208  \
                             afg        0.000   22.372   37.237   47.532   
                             ago       34.484   55.315   98.686  127.389   
                             alb       14.743   19.170   24.545   26.838   
                             are      209.548  272.055  318.142  453.516   

                                         2020     2025     2030      2035   
model         scenario       country                                        
REMIND-MAGPIE SSP5-34-OS-V27 abw        0.277    0.395    0.589     0.885  \
                             afg       58.848   79.059  114.303   171.645   
                             ago      173.632  214.748  264.116   323.521   
                             alb       30.302   35.462   42.765    51.992   
                             are      567.148  720.458  921.726  1160.236   

                                          2040      2045  ...      2055   
model         scenario       country                      ...             
REMIND-MAGPIE SSP5-34-OS-V27 abw         1.291     1.823  ...     3.327  \
                             afg       257.527   380.116  ...   767.620   
                             ago       401.276   510.353  ...   869.382   
                             alb        62.170    72.826  ...    93.510   
                             are      1414.170  1658.828  ...  2073.427   

                                          2060      2065      2070      2075   
model         scenario       country                                           
REMIND-MAGPIE SSP5-34-OS-V27 abw         4.318     5.468     6.769     8.208  \
                             afg      1044.537  1383.535  1785.195  2249.365   
                             ago      1133.130  1456.094  1833.656  2260.130   
                             alb       103.790   113.536   121.975   129.225   
                             are      2269.341  2449.734  2606.055  2727.411   

                                          2080      2085      2090      2095   
model         scenario       country                                           
REMIND-MAGPIE SSP5-34-OS-V27 abw         9.770    11.351    12.921    14.432  \
                             afg      2776.635  3362.675  4004.527  4700.203   
                             ago      2727.862  3227.709  3752.107  4292.043   
                             alb       135.404   140.236   143.913   146.697   
                             are      2826.731  2907.985  2972.880  3018.047   

                                          2100  
model         scenario       country            
REMIND-MAGPIE SSP5-34-OS-V27 abw        15.810  
                             afg      5442.728  
                             ago      4834.886  
                             alb       148.263  
                             are      3029.692  

[5 rows x 21 columns]

In [71]:
downscaler = Downscaler(
    harmonized.pix.semijoin(variabledefs.index_regional, how="inner"),
    hist.pix.semijoin(variabledefs.index_regional, how="inner"),
    base_year,
    regionmapping.data,
    gdp=gdp,
)
results = downscaler.downscale()

/Users/coroa/repos/aneris/src/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))
/Users/coroa/repos/aneris/src/aneris/downscaling/intensity_convergence.py:328: RuntimeWarning: invalid value encountered in multiply
  (


In [72]:
downscaler.methods().value_counts()

/Users/coroa/repos/aneris/src/aneris/methods.py:389: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.std(x) / np.mean(x))


method
ipat_2100_gdp        781
proxy_gdp            132
base_year_pattern     33
Name: count, dtype: int64

In [73]:
results

year                                                                                                  2015   
gas    sector                              region unit      model         scenario       country             
CH4    Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.220823  \
                                                                                         bdi      0.036465   
                                                                                         ben      0.095516   
                                                                                         bfa      0.511754   
                                                                                         bwa      0.105129   
...                                                                                                    ...   
Sulfur Agriculture                         USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 usa      0.000000   
       Solvents Production and Application USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   
VOC    Agriculture                         USA    Mt VOC/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   

year                                                                                                  2020   
gas    sector                              region unit      model         scenario       country             
CH4    Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.279381  \
                                                                                         bdi      0.046135   
                                                                                         ben      0.120845   
                                                                                         bfa      0.647461   
                                                                                         bwa      0.133007   
...                                                                                                    ...   
Sulfur Agriculture                         USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 usa      0.000000   
       Solvents Production and Application USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   
VOC    Agriculture                         USA    Mt VOC/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                                                                         usa      0.000000   

year                                                                                                  2030   
gas    sector                              region unit      model         scenario       country             
CH4    Agriculture                         AFR    Mt CH4/yr REMIND-MAGPIE SSP5-34-OS-V27 ago      0.515045  \
                                                                                         bdi      0.085051   
                                                                                         ben      0.222780   
                                                                                         bfa      1.193611   
                                                                                         bwa      0.245201   
...                                                                                                    ...   
Sulfur Agriculture                         USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 usa      0.000000   
       Solvents Production and Application USA    Mt SO2/yr REMIND-MAGPIE SSP5-34-OS-V27 pri      0.000000   
                                      

# Gridding

In [19]:
idxr = xr.open_dataarray(base_path / "gridding_process_files" / "iso_mask.nc", chunks={"iso": 20}).rename({"iso": "country"})

In [ ]:
proxy_dir = base_path / "gridding_process_files" / "proxy_rasters"
proxy_cfg = pd.concat(
    [
        # DataFrame(
        #     {
        #         "path": proxy_dir.glob("aircraft_*.nc"),
        #         "name": "em-AIR-anthro",
        #         "separate_shares": False,
        #     }
        # ),
        DataFrame(
            {
                "path": proxy_dir.glob("anthro_*.nc"),
                "name": "em-anthro",
                "separate_shares": False,
            }
        ),
        # DataFrame(
        #     {
        #         "path": proxy_dir.glob("openburning_*.nc"),
        #         "name": "em-openburning",
        #         "separate_shares": True,
        #     }
        # ),
        DataFrame({"path": proxy_dir.glob("shipping_*.nc"), "name": ..., "template": ..., "separate_shares": False})
    ]
).assign(
    name=lambda df: df.path.map(lambda p: p.stem.split("_")[1]) + "-" + df.name,
    template="{name}_emissions_{model}-{scenario}_201501-210012",
)
proxy_cfg.head()

,path,name,separate_shares,template
0,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,OC-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
1,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,CO-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
2,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,NH3-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
3,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,NOx-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012
4,/Users/coroa/Library/CloudStorage/OneDrive-Sha...,CO2-em-anthro,False,{name}_emissions_{model}-{scenario}_201501-210012


In [21]:
sector_mapping = {
    "AIR": "Aircraft",
    "SHP": "International Shipping",
    "AWB": "Agricultural Waste Burning",
    "AGR": "Agriculture",
    "ENE": "Energy Sector",
    "FRTB": "Forest Burning",
    "GRSB": "Grassland Burning",
    "IND": "Industrial Sector",
    "PEAT": "Peat Burning",
    "RCO": "Residential Commercial Other",
    "SLV": "Solvents Production and Application",
    "TRA": "Transportation Sector",
    "WST": "Waste",
}

In [42]:
kg_per_mt = 1e9
s_per_yr = 365 * 24 * 60 * 60
results = (
    results.rename(index=lambda s: re.sub("Mt (.*)/yr", r"kg \1/s", s), level="unit")
    * kg_per_mt
    / s_per_yr
)

In [43]:
results = results.droplevel("region")
results.head()

year                                                                2015   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       7.002247  \
                                                      bdi       1.156298   
                                                      ben       3.028788   
                                                      bfa      16.227619   
                                                      bwa       3.333608   

year                                                                2020   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       8.859105  \
                                                      bdi       1.462925   
                                                      ben       3.831963   
                                                      bfa      20.530863   
                                                      bwa       4.217615   

year                                                                2030   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      16.331986  \
                                                      bdi       2.696940   
                                                      ben       7.064321   
                                                      bfa      37.849168   
                                                      bwa       7.775281   

year                                                                2040   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      27.781679  \
                                                      bdi       4.587656   
                                                      ben      12.016831   
                                                      bfa      64.383687   
                                                      bwa      13.226216   

year                                                                2050   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      24.593980  \
                                                      bdi       4.061263   
                                                      ben      10.638007   
                                                      bfa      56.996234   
                                                      bwa      11.708626   

year                                                                2060   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago      12.604726  \
                                                      bdi       2.081449   
                                                      ben       5.452113   
                                                      bfa      29.211290   
                                                      bwa       6.000819   

year                                                                2070   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       5.649102  \
                                                      bdi       0.932850   
                                                      ben       2.443492   
                                                      bfa      13.091722   
                                                      bwa       2.689407   

year                                                                2080   
gas sector      unit     model         scenario       country              
CH4 Agriculture kg CH4/s REMIND-MAGPIE SSP5-34-OS-V27 ago       4.963229  \
                                         

In [45]:
gridder = Gridder(results, idxr, proxy_cfg, index_mappings=dict(sector=sector_mapping), output_dir="../results")

In [50]:
gridder.grid(skip_check=True)

INFO:root:Collecting tasks for proxy OC-em-anthro
/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(
INFO:root:Adding tasks for {'unit': 'kg OC/s', 'model': 'REMIND-MAGPIE', 'scenario': 'SSP5-34-OS-V27'}
INFO:root:Writing to ../results/OC-em-anthro_emissions_REMIND-MAGPIE-SSP5-34-OS-V27_201501-210012.nc
INFO:root:Adding tasks for {'unit': 'kg OC/s', 'model': 'IMAGE', 'scenario': 'SSP2'}
INFO:root:Writing to ../results/OC-em-anthro_emissions_IMAGE-SSP2_201501-210012.nc


[                                        ] | 0% Completed | 8.98 s ms

/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 12m 58s


INFO:root:Collecting tasks for proxy CO-em-anthro
/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 12
  result = blockwise(
INFO:root:Adding tasks for {'unit': 'kg CO/s', 'model': 'REMIND-MAGPIE', 'scenario': 'SSP5-34-OS-V27'}
INFO:root:Writing to ../results/CO-em-anthro_emissions_REMIND-MAGPIE-SSP5-34-OS-V27_201501-210012.nc
INFO:root:Adding tasks for {'unit': 'kg CO/s', 'model': 'IMAGE', 'scenario': 'SSP2'}
INFO:root:Writing to ../results/CO-em-anthro_emissions_IMAGE-SSP2_201501-210012.nc


[                                        ] | 0% Completed | 7.61 s ms

/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[##################                      ] | 45% Completed | 393.95 s


KeyboardInterrupt: 